In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm

In [2]:
training_indexes = np.load('../../sorted_data/LFP/training/index_training_LFP.npy').astype(np.int)
df = pd.DataFrame(training_indexes)
df.reset_index(inplace=True)
df.columns = ['trial','image_id']
availabe_image_ids = df.image_id.unique()
train_size = int(len(availabe_image_ids)*0.9)
np.random.seed(7)
train_image_ids = np.random.choice(availabe_image_ids, size=train_size, replace=False)
train_image_ids_series = pd.Series('train', index=train_image_ids, name='train_or_test')
df = df.join(train_image_ids_series, on='image_id')
df.train_or_test.fillna('test',inplace=True)

### Training vs Testing df

In [3]:
test_df = df[df["train_or_test"] == "test"]
train_df = df[df["train_or_test"] == "train"]

### Seen Images

INDICES:

In [6]:
test_indexes = test_df.image_id
train_indexes = train_df.image_id
np.save('../../sorted_data/LFP/testing/index_testing_LFP_split.npy', np.array(test_indexes) )
np.save('../../sorted_data/LFP/training/index_training_LFP_split.npy', np.array(train_indexes) )

In [18]:
def crop_images(set_df):
    
    '''
    
    set_df can be either test_df or train_df,
    which is dataframe with an image_id column
    representing image_ids with respect to trials.
    Returns a list of cropped images
    '''
    
    img_dir = f'../../DeepMonkey/Data/Stimuli/Processed/Training/'
    cropped_img = []
    for i in tqdm(set_df.image_id.unique()):
        t = plt.imread(img_dir+str(i).zfill(4)+'.png')

        img_width = t.shape[0]
        left_e = int((img_width / 100 * 27.5))
        right_s = int(img_width - (img_width/100*32.5))

        img_height = t.shape[1]
        top_e = int((img_height / 100 * 27.5))
        bot_s = int(img_height - (img_height/100*32.5))

        cropped_t = t[left_e:right_s, top_e:bot_s, :]
        cropped_img.append(cropped_t)
    return cropped_img


In [19]:
test_cropped_img = crop_images(test_df)
train_cropped_img = crop_images(train_df)
np.save('splitcrop_testing_im.npy', test_cropped_img)
np.save('splitcrop_training_im.npy', train_cropped_img)

100%|██████████| 7998/7998 [03:21<00:00, 39.70it/s]


### LFP signals: (electrodes, ms, trial)

In [9]:
nn_training_old = np.load('../../sorted_data/LFP/training/LFP_training_data32_normalized_fast.npy')

In [10]:
nn_training_old.shape

(191, 916, 27349)

In [11]:
nn_test =  nn_training_old[:,:,test_df.trial.values]
nn_train =  nn_training_old[:,:,train_df.trial.values]

In [14]:
np.save(f'../../sorted_data/LFP/training/LFP_training_splitted.npy',nn_train)


In [15]:
np.save(f'../../sorted_data/LFP/testing/LFP_testing_splitted.npy',nn_test)

In [ ]:
# ! scp lynnle@lovelace.ai.ru.nl:/scratch/users/lynnle/MonkeyProject/git/sorted_data/LFP/training/LFP_training_splitted.npy /huge/lynnle/MonkeyProject/git/sorted_data/LFP/training/